## 1. Setup & Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import shapiro, normaltest, spearmanr, pearsonr
import warnings
warnings.filterwarnings('ignore')

# Set style for publication-quality plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✓ All imports successful")

## 2. Load & Inspect Data

In [ ]:
# For now, create synthetic data that mirrors behavioral study structure
# In production: Load actual thesis data with df = pd.read_csv('data/raw/behavioral_survey.csv')

np.random.seed(42)
n_subjects = 194

# Synthetic behavioral data mimicking cognitive assessments
data = pd.DataFrame({
    'subject_id': range(1, n_subjects + 1),
    'crt_score': np.random.normal(loc=1.5, scale=0.8, size=n_subjects).clip(0, 3),
    'nfc_score': np.random.normal(loc=3.8, scale=0.7, size=n_subjects).clip(1, 5),
    'conspiracy_mentality': np.random.normal(loc=2.8, scale=0.9, size=n_subjects).clip(1, 5),
    'bs_receptivity': np.random.normal(loc=2.5, scale=1.0, size=n_subjects).clip(0, 5),
    'rational_style': np.random.normal(loc=3.5, scale=0.8, size=n_subjects).clip(1, 5),
    'intuitive_style': np.random.normal(loc=3.2, scale=0.9, size=n_subjects).clip(1, 5),
    'verification_behavior': np.random.normal(loc=3.0, scale=1.1, size=n_subjects).clip(0, 5),
    'age': np.random.normal(loc=35, scale=12, size=n_subjects).clip(18, 75).astype(int),
    'education_years': np.random.normal(loc=14, scale=2.5, size=n_subjects).clip(8, 20).astype(int),
    'misinformation_susceptibility': np.random.normal(loc=2.5, scale=1.2, size=n_subjects).clip(0, 5)
})

# Add some missingness (realistic 5-10%)
missing_indices = np.random.choice(len(data), size=int(0.07 * len(data)), replace=False)
data.loc[missing_indices, 'bs_receptivity'] = np.nan

print(f"✓ Loaded synthetic behavioral data: {data.shape[0]} subjects, {data.shape[1]} features")
print(f"\nFirst 5 rows:")
print(data.head())
print(f"\nData types:\n{data.dtypes}")

## 3. Data Quality Assessment

In [ ]:
# Check for missing data
missing_summary = pd.DataFrame({
    'feature': data.columns,
    'missing_count': data.isnull().sum(),
    'missing_percent': (data.isnull().sum() / len(data) * 100).round(2)
})

print("Missing Data Summary:")
print(missing_summary[missing_summary['missing_count'] > 0])

# Duplicate check
print(f"\nDuplicate rows: {data.duplicated().sum()}")

# Summary statistics
print(f"\nBasic Statistics:")
print(data.describe().round(3))

## 4. Descriptive Statistics for Cognitive Features

In [ ]:
# Focus on cognitive/behavioral features (excluding demographics)
cognitive_features = [
    'crt_score', 'nfc_score', 'conspiracy_mentality', 'bs_receptivity',
    'rational_style', 'intuitive_style', 'verification_behavior',
    'misinformation_susceptibility'
]

# Create comprehensive descriptive table
descriptive_stats = pd.DataFrame({
    'Mean': data[cognitive_features].mean().round(3),
    'SD': data[cognitive_features].std().round(3),
    'Median': data[cognitive_features].median().round(3),
    'Min': data[cognitive_features].min().round(3),
    'Max': data[cognitive_features].max().round(3),
    'Skewness': data[cognitive_features].skew().round(3),
    'Kurtosis': data[cognitive_features].kurtosis().round(3),
    'N': data[cognitive_features].count()
})

print("Table 1: Descriptive Statistics for Cognitive Features")
print("="*90)
print(descriptive_stats)
print("\nInterpretation:")
print("- Skewness > 0 = right-skewed; < 0 = left-skewed")
print("- Kurtosis > 0 = heavier tails (leptokurtic); < 0 = lighter tails (platykurtic)")

## 5. Normality Testing

In [ ]:
# Test normality using Shapiro-Wilk (appropriate for N < 300)
normality_results = []

for feature in cognitive_features:
    # Remove NaN for testing
    valid_data = data[feature].dropna()
    
    # Shapiro-Wilk test
    stat, p_value = shapiro(valid_data)
    is_normal = 'Yes' if p_value > 0.05 else 'No'
    
    normality_results.append({
        'Feature': feature,
        'Shapiro-Wilk Stat': round(stat, 4),
        'p-value': round(p_value, 4),
        'Normal (α=0.05)': is_normal
    })

normality_df = pd.DataFrame(normality_results)

print("Table 2: Shapiro-Wilk Normality Test")
print("="*80)
print(normality_df.to_string(index=False))
print("\nNote: p-value > 0.05 suggests distribution is consistent with normality.")
print(f"Non-normal features: {len(normality_df[normality_df['Normal (α=0.05)'] == 'No'])} / {len(normality_df)}")

## 6. Distribution Visualizations

In [ ]:
# Create histograms with normal distribution overlays
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for idx, feature in enumerate(cognitive_features):
    ax = axes[idx]
    
    # Histogram
    ax.hist(data[feature].dropna(), bins=20, density=True, alpha=0.7, color='steelblue', edgecolor='black')
    
    # Overlay normal distribution
    mu, sigma = data[feature].mean(), data[feature].std()
    x = np.linspace(data[feature].min(), data[feature].max(), 100)
    ax.plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal fit')
    
    ax.set_title(f'{feature}\n(μ={mu:.2f}, σ={sigma:.2f})', fontsize=10, fontweight='bold')
    ax.set_xlabel('Score')
    ax.set_ylabel('Density')
    ax.legend(fontsize=8)
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('docs/figures/01_feature_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: docs/figures/01_feature_distributions.png")

## 7. Q-Q Plots for Normality Assessment

In [ ]:
# Q-Q plots for visual normality assessment
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for idx, feature in enumerate(cognitive_features):
    ax = axes[idx]
    
    # Remove NaN
    valid_data = data[feature].dropna()
    
    # Q-Q plot
    stats.probplot(valid_data, dist="norm", plot=ax)
    ax.set_title(f'{feature}', fontsize=10, fontweight='bold')
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('docs/figures/02_qq_plots.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: docs/figures/02_qq_plots.png")

## 8. Correlation Analysis

In [ ]:
# Pearson correlation (parametric)
pearson_corr = data[cognitive_features].corr(method='pearson')

print("Pearson Correlation Matrix (cognitive features):")
print("="*80)
print(pearson_corr.round(3))

# Spearman correlation (non-parametric, more robust)
spearman_corr = data[cognitive_features].corr(method='spearman')

print("\nSpearman Correlation Matrix (cognitive features):")
print("="*80)
print(spearman_corr.round(3))

## 9. Correlation Heatmaps

In [ ]:
# Pearson heatmap
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(pearson_corr, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax,
            vmin=-1, vmax=1)
ax.set_title('Pearson Correlation Matrix: Cognitive Features', fontsize=12, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('docs/figures/03_pearson_correlation.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: docs/figures/03_pearson_correlation.png")

# Spearman heatmap
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(spearman_corr, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax,
            vmin=-1, vmax=1)
ax.set_title('Spearman Correlation Matrix: Cognitive Features', fontsize=12, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('docs/figures/04_spearman_correlation.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: docs/figures/04_spearman_correlation.png")

## 10. Key Correlations with Misinformation Susceptibility

In [ ]:
# Focus on correlations with target outcome
target = 'misinformation_susceptibility'
predictor_features = [f for f in cognitive_features if f != target]

correlation_with_target = []

for feature in predictor_features:
    # Pearson
    pearson_r, pearson_p = pearsonr(data[feature].dropna(), data[target][:len(data[feature].dropna())])
    
    # Spearman (more robust)
    spearman_r, spearman_p = spearmanr(data[feature].dropna(), data[target][:len(data[feature].dropna())])
    
    correlation_with_target.append({
        'Feature': feature,
        'Pearson r': round(pearson_r, 3),
        'Pearson p': round(pearson_p, 4),
        'Spearman ρ': round(spearman_r, 3),
        'Spearman p': round(spearman_p, 4)
    })

target_corr_df = pd.DataFrame(correlation_with_target).sort_values('Spearman ρ', ascending=False, key=abs)

print(f"Table 3: Correlations with {target}")
print("="*100)
print(target_corr_df.to_string(index=False))
print("\nSignificant correlations (p < 0.05):")
print(target_corr_df[target_corr_df['Spearman p'] < 0.05])

## 11. Multivariate Structure & Dimensionality

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Standardize features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(data[cognitive_features].dropna())

# PCA
pca = PCA()
pca.fit(features_scaled)

# Explained variance
cumsum_var = np.cumsum(pca.explained_variance_ratio_)

print("Explained Variance by Principal Component:")
print("="*60)
for i in range(min(8, len(pca.explained_variance_ratio_))):
    print(f"PC{i+1}: {pca.explained_variance_ratio_[i]:.1%} (Cumulative: {cumsum_var[i]:.1%})")

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
ax1.bar(range(1, len(pca.explained_variance_ratio_)+1), pca.explained_variance_ratio_, alpha=0.7, color='steelblue')
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Explained Variance Ratio')
ax1.set_title('Scree Plot', fontweight='bold')
ax1.grid(alpha=0.3)

# Cumulative variance
ax2.plot(range(1, len(cumsum_var)+1), cumsum_var, 'bo-', linewidth=2, markersize=6)
ax2.axhline(y=0.95, color='r', linestyle='--', label='95% threshold')
ax2.set_xlabel('Number of Components')
ax2.set_ylabel('Cumulative Explained Variance')
ax2.set_title('Cumulative Explained Variance', fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('docs/figures/05_pca_variance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Saved: docs/figures/05_pca_variance.png")

## 12. Summary & Validation Report

In [ ]:
print("""
="*80
BEHAVIORAL ANALYSIS: EXPLORATORY DATA ANALYSIS REPORT
="*80

SAMPLE CHARACTERISTICS:
-" * 80)
print(f"Sample size: N = {len(data)}")
print(f"Age: M = {data['age'].mean():.1f}, SD = {data['age'].std():.1f}, Range = {data['age'].min()}-{data['age'].max()}")
print(f"Education: M = {data['education_years'].mean():.1f} years, SD = {data['education_years'].std():.1f}")

print(f"""
DATA QUALITY:
-" * 80)
print(f"Missing data: {data.isnull().sum().sum()} values ({data.isnull().sum().sum() / (data.shape[0] * data.shape[1]) * 100:.1f}%)")
print(f"Duplicates: 0")

print(f"""
NORMALITY:
-" * 80)
normal_count = len(normality_df[normality_df['Normal (α=0.05)'] == 'Yes'])
print(f"Features with normal distribution: {normal_count}/{len(normality_df)}")
print(f"Non-normal features: {normality_df[normality_df['Normal (α=0.05)'] == 'No']['Feature'].tolist()}")
print("\nRecommendation: Use both parametric (Pearson) and non-parametric (Spearman) methods.")

print(f"""
KEY CORRELATIONS WITH MISINFORMATION SUSCEPTIBILITY:
-" * 80)
sig_correlations = target_corr_df[target_corr_df['Spearman p'] < 0.05]
if len(sig_correlations) > 0:
    for idx, row in sig_correlations.iterrows():
        print(f"  - {row['Feature']}: ρ = {row['Spearman ρ']:.3f}, p = {row['Spearman p']:.4f}")
else:
    print("  No statistically significant correlations at α = 0.05")

print(f"""
DIMENSIONALITY:
-" * 80)
n_components_95 = np.argmax(cumsum_var >= 0.95) + 1
print(f"Components needed for 95% variance explained: {n_components_95}")
print(f"Total features: {len(cognitive_features)}")
print(f"Dimensionality reduction potential: Yes, consider PCA or feature selection")

print(f"""
NEXT STEPS:
-" * 80)
print("1. Handle missing data (imputation or deletion)")
print("2. Create feature composites if warranted by theory")
print("3. Standardize features for regression modeling")
print("4. Proceed to feature engineering (02_feature_engineering.ipynb)")
print(f"\n{'='*80}")
""")

## 13. Export Clean Dataset for Next Steps

In [ ]:
# Save cleaned dataset
data_clean = data.dropna()
data_clean.to_csv('data/processed/behavioral_features_clean.csv', index=False)

print(f"✓ Saved clean dataset: {len(data_clean)} subjects, {len(data_clean.columns)} features")
print(f"  Path: data/processed/behavioral_features_clean.csv")

# Save descriptive statistics for manuscript
descriptive_stats.to_csv('behavioral_analysis/results/01_descriptive_statistics.csv')
print(f"\n✓ Saved descriptive statistics: behavioral_analysis/results/01_descriptive_statistics.csv")

# Save normality results
normality_df.to_csv('behavioral_analysis/results/02_normality_tests.csv', index=False)
print(f"✓ Saved normality results: behavioral_analysis/results/02_normality_tests.csv")